In [1]:
ls


 Volume in drive C is Windows 
 Volume Serial Number is 04F5-FC96

 Directory of C:\Users\Yamlak\Documents\IU PhD Journey\ML for BioInformatics\Multifactor Analysis of Prostate Cancer

22/03/2025  10:27    <DIR>          .
22/03/2025  09:46    <DIR>          ..
22/03/2025  10:27    <DIR>          .ipynb_checkpoints
22/03/2025  10:27                72 Analysis.ipynb
22/03/2025  10:21       127,450,102 data_curated_trans.csv
               2 File(s)    127,450,174 bytes
               3 Dir(s)  10,952,663,040 bytes free


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [58]:
data = pd.read_csv('data_curated_trans.csv')

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489 entries, 0 to 488
Columns: 45606 entries, PATIENT_ID to hsa-mir-224
dtypes: float64(45605), object(1)
memory usage: 170.1+ MB


In [60]:
print("Dataset Shape:", data.shape)  # (rows, columns)
print("\nFirst 5 rows of the dataset:")
print(data.head())


num_columns = data.shape[1]
print(f"\nTotal number of columns: {num_columns}")

Dataset Shape: (489, 45606)

First 5 rows of the dataset:
     PATIENT_ID  PATH_N_STAGE  NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT  \
0  TCGA-2A-A8VL           2.0                                      2.0   
1  TCGA-2A-A8VO           3.0                                      2.0   
2  TCGA-2A-A8VT           1.0                                      2.0   
3  TCGA-2A-A8VV           2.0                                      2.0   
4  TCGA-2A-A8VX           2.0                                      2.0   

   PATH_T_STAGE  PRIOR_DX  RADIATION_THERAPY   AGE  M_UBE2Q2P2  M_HMGB1P1  \
0           1.0       2.0                2.0  51.0     -0.0361        NaN   
1           2.0       2.0                2.0  57.0     -0.3004        NaN   
2           2.0       2.0                1.0  47.0      3.1758        NaN   
3           1.0       2.0                2.0  52.0     -0.1102        NaN   
4           2.0       2.0                2.0  70.0     -0.7052        NaN   

   M_LOC155060  ...  hsa-mir-1321 

In [61]:
missing_values = data.isnull().sum()
missing_values 

PATIENT_ID                                 0
PATH_N_STAGE                               0
NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT    0
PATH_T_STAGE                               0
PRIOR_DX                                   0
                                          ..
hsa-mir-513c                               0
hsa-mir-513b                               0
hsa-mir-513a-1                             0
hsa-mir-513a-2                             0
hsa-mir-224                                0
Length: 45606, dtype: int64

In [62]:
missing_values.sum()

281754

In [63]:

# Filter columns that have missing values
columns_with_missing = missing_values[missing_values > 0]

# Display missing values
print("\nColumns with missing values and their count:")
print(columns_with_missing)



Columns with missing values and their count:
M_HMGB1P1     489
M_RNU12-2P    489
M_SSX9         33
M_CXORF67     489
M_EFCAB8      489
             ... 
M_ZNF679       33
M_ZNF735       33
M_ZNF99        33
M_ZP4          33
M_ZSWIM2       33
Length: 1546, dtype: int64


In [64]:
data_cleaned = data.dropna(axis=1)

In [65]:
data.shape, data_cleaned.shape

((489, 45606), (489, 44060))

In [66]:
data_cleaned.isnull().sum().sum()

0

In [67]:
numerical_cols = data_cleaned.select_dtypes(include=['number']).columns

num_features = len(numerical_cols)
features_per_plot = 20
num_plots = int(np.ceil(num_features / features_per_plot))

# Generate box plots in batches of 20 features
# for i in range(num_plots):
#     plt.figure(figsize=(15, 6))
    

#     subset_cols = numerical_cols[i * features_per_plot : (i + 1) * features_per_plot]
    

#     data_cleaned[subset_cols].boxplot()
#     plt.xticks(rotation=90)
#     plt.title(f"Boxplot for Features {i * features_per_plot + 1} to {(i + 1) * features_per_plot}")
#     plt.show()

In [68]:
data_cleaned.columns 

Index(['PATIENT_ID', 'PATH_N_STAGE', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT',
       'PATH_T_STAGE', 'PRIOR_DX', 'RADIATION_THERAPY', 'AGE', 'M_UBE2Q2P2',
       'M_LOC155060', 'M_GTPBP6',
       ...
       'hsa-mir-1321', 'hsa-mir-361', 'hsa-mir-548m', 'hsa-mir-652',
       'hsa-mir-220a', 'hsa-mir-513c', 'hsa-mir-513b', 'hsa-mir-513a-1',
       'hsa-mir-513a-2', 'hsa-mir-224'],
      dtype='object', length=44060)

In [69]:
exclude_columns = ['PATIENT_ID', 'PATH_N_STAGE', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT', 
                   'PATH_T_STAGE', 'PRIOR_DX', 'RADIATION_THERAPY', 'AGE']

# Exclude the specified columns
data_filtered = data_cleaned.drop(columns=exclude_columns)

In [70]:
exclude_columns = ['PATIENT_ID', 'PATH_N_STAGE', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT', 
                   'PATH_T_STAGE', 'PRIOR_DX', 'RADIATION_THERAPY', 'AGE']

# Remove the excluded columns before computing the standard deviation
data_filtered_for_sd = data_cleaned.drop(columns=exclude_columns)

# Calculate the standard deviation for the remaining columns
std_dev = data_filtered_for_sd.std()

# Set a threshold to filter out high-variability columns (optional: you can adjust the threshold)
threshold = std_dev.mean() + 1 * std_dev.std()  # Keep columns with SD below (mean + 1*SD)

# Get the filtered columns based on the threshold
filtered_columns = std_dev[std_dev < threshold].index

# Create a new dataset with only the selected columns (filtered columns + excluded columns)
data_filtered = data_cleaned[exclude_columns + filtered_columns.tolist() ]

# Display the number of features before and after filtering
print(f"Original number of features: {data_cleaned.shape[1]}")
print(f"Number of features after filtering: {data_filtered.shape[1]}")


# Show the filtered columns
print("\nColumns with smaller variability (excluding excluded ones):")
print(filtered_columns)

Original number of features: 44060
Number of features after filtering: 43964

Columns with smaller variability (excluding excluded ones):
Index(['M_UBE2Q2P2', 'M_LOC155060', 'M_GTPBP6', 'M_EFCAB12', 'M_A1BG',
       'M_A1CF', 'M_RBFOX1', 'M_GGACT', 'M_A2M', 'M_A2ML1',
       ...
       'hsa-mir-1321', 'hsa-mir-361', 'hsa-mir-548m', 'hsa-mir-652',
       'hsa-mir-220a', 'hsa-mir-513c', 'hsa-mir-513b', 'hsa-mir-513a-1',
       'hsa-mir-513a-2', 'hsa-mir-224'],
      dtype='object', length=43957)


In [71]:
data_filtered[filtered_columns]

,M_UBE2Q2P2,M_LOC155060,M_GTPBP6,M_EFCAB12,M_A1BG,M_A1CF,M_RBFOX1,M_GGACT,M_A2M,M_A2ML1,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,-0.0361,0.3014,-0.2867,0.9115,0.2252,-0.1838,0.3830,-0.5284,-0.7255,0.1866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.3004,0.6841,0.6619,1.7160,-0.6237,-0.1838,0.1567,-0.0413,0.4708,-0.4744,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.1758,2.8191,0.2520,-0.6433,-0.6380,-0.1633,-0.3436,-0.1908,-0.7193,-0.4985,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-0.1102,-0.1719,-0.2612,0.2185,-0.6294,-0.1838,-0.2837,0.0488,-0.4417,-0.5010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.7052,0.3580,0.9095,-0.1748,-0.1893,-0.1838,-0.3850,1.3400,-0.4493,-0.4765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,0.6768,10.6747,-0.9893,0.0690,-0.3671,-0.1838,-0.3078,-1.8237,-0.4308,0.4972,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0
485,-0.2252,-0.0288,-0.1738,-1.3305,0.1966,-0.1838,0.0035,-0.6731,0.7859,0.5392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,-0.8350,-0.5767,0.5719,-1.2691,-0.6668,-0.1838,-0.4006,-0.5881,0.0745,-0.2450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
487,0.4274,-0.0456,0.0093,-0.0881,-0.3998,-0.1838,-0.4006,2.6657,-0.7081,-0.0225,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the selected filtered columns
data_filtered[filtered_columns] = scaler.fit_transform(data_filtered[filtered_columns])

# Display the first few rows of the normalized data
print(data_filtered[filtered_columns].head())


<ipython-input-72-fe714ff2cc52>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered[filtered_columns] = scaler.fit_transform(data_filtered[filtered_columns])


In [73]:
filtered_columns

Index(['M_UBE2Q2P2', 'M_LOC155060', 'M_GTPBP6', 'M_EFCAB12', 'M_A1BG',
       'M_A1CF', 'M_RBFOX1', 'M_GGACT', 'M_A2M', 'M_A2ML1',
       ...
       'hsa-mir-1321', 'hsa-mir-361', 'hsa-mir-548m', 'hsa-mir-652',
       'hsa-mir-220a', 'hsa-mir-513c', 'hsa-mir-513b', 'hsa-mir-513a-1',
       'hsa-mir-513a-2', 'hsa-mir-224'],
      dtype='object', length=43957)

In [74]:
data_filtered.columns

Index(['PATIENT_ID', 'PATH_N_STAGE', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT',
       'PATH_T_STAGE', 'PRIOR_DX', 'RADIATION_THERAPY', 'AGE', 'M_UBE2Q2P2',
       'M_LOC155060', 'M_GTPBP6',
       ...
       'hsa-mir-1321', 'hsa-mir-361', 'hsa-mir-548m', 'hsa-mir-652',
       'hsa-mir-220a', 'hsa-mir-513c', 'hsa-mir-513b', 'hsa-mir-513a-1',
       'hsa-mir-513a-2', 'hsa-mir-224'],
      dtype='object', length=43964)

# Progression

In [75]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

X = data_filtered.drop(columns= ['PATIENT_ID', 'PATH_N_STAGE'])  # All features
y = data_filtered['PATH_N_STAGE']  # Replace with the appropriate target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit LassoCV
lasso = LassoCV(cv=5)  # Lasso with cross-validation
lasso.fit(X_train, y_train)

# Get the features selected by Lasso (non-zero coefficients)
selected_features = X.columns[(lasso.coef_ != 0)]

print(f"Selected features by Lasso: {selected_features}")


Selected features by Lasso: Index(['AGE', 'M_AJAP1', 'M_ALPI', 'M_BARX1', 'M_BTBD18', 'M_TMEM52B',
       'M_DHRS4-AS1', 'M_PRR35', 'M_SPATA46', 'M_C1ORF115', 'M_SPANXC.4',
       'M_C20ORF203', 'M_C9ORF163', 'M_CARS', 'M_CC2D2A', 'M_CCL8', 'M_CHRNA6',
       'M_CSTL1', 'M_DDX5', 'M_DNAH8', 'M_FAF2', 'M_FAM19A1', 'M_FAM25A',
       'M_FAM76A', 'M_FAR1', 'M_FLI1', 'M_FMOD', 'M_FSCB', 'M_GAL3ST3',
       'M_HERC5', 'M_HIST1H3A', 'M_HIST1H3J', 'M_ARHGAP45', 'M_IL1R1',
       'M_KCNJ13', 'M_KLC3', 'M_ASH1L-AS1', 'M_LSM11', 'M_MAP2K6', 'M_MMD2',
       'M_NRP1', 'M_OMP', 'M_OR1J1', 'M_PCSK7', 'M_HELZ2', 'M_RNF213',
       'M_SAMD9', 'M_SPOP', 'M_SRI', 'M_UNC93A', 'M_ZNF22', 'SCARNA3',
       'HNRNPLL', 'EGFEM1P', 'LOC101927069', 'PRKACG', 'FXN', 'KLF4',
       'SLC22A24', 'SLC22A25', 'MIR3680-1', 'MIR4300', 'B4GALNT3', 'AWAT2',
       'MIR676'],
      dtype='object')


In [79]:
selected_features_for_progression = selected_features

In [80]:
len(selected_features_for_progression), 

(65,)

# Therapy Response

In [81]:

from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

X = data_filtered.drop(columns= ['PATIENT_ID', 'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT'])  # All features
y = data_filtered['NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT']  # Replace with the appropriate target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit LassoCV
lasso = LassoCV(cv=5)  # Lasso with cross-validation
lasso.fit(X_train, y_train)

# Get the features selected by Lasso (non-zero coefficients)
selected_features = X.columns[(lasso.coef_ != 0)]

print(f"Selected features by Lasso: {selected_features}")



Selected features by Lasso: Index(['M_ADORA3', 'M_EBPL', 'M_ENSA', 'M_FEZF1', 'M_HYAL4', 'M_KCNK9',
       'M_KRTAP5-1', 'M_PP7080', 'M_PLEKHN1', 'M_RELL2', 'M_SRGAP2', 'M_TREM2',
       'M_UMOD', 'GATAD2B', 'RP11-463J7.2', 'SRGAP2B', 'SRGAP2C',
       'RP13-507P19.2', 'REM2', 'RBM23', 'VPS13C', 'C2CD4A', 'CHTF18', 'GNG13',
       'MIR662', 'PPM1F', 'LIF', 'SF3A1'],
      dtype='object')


In [83]:
selected_features_for_response = selected_features

In [84]:
len(selected_features_for_response)

28